# Exercise E: Combined applications

In [14]:
from typing import Sequence
from aleph_alpha_client import ImagePrompt, AlephAlphaClient, AlephAlphaModel, SemanticEmbeddingRequest, SemanticRepresentation, Prompt, SummarizationRequest, CompletionRequest, EvaluationRequest
import math
import os

In [4]:
# instantiate the client and model
search_model = AlephAlphaModel(
    AlephAlphaClient(host="https://api.aleph-alpha.com", token=os.getenv("API_TOKEN")),
    model_name = "luminous-base"
)

model = AlephAlphaModel(
    AlephAlphaClient(host="https://api.aleph-alpha.com", token=os.getenv("API_TOKEN")),
    model_name = "luminous-extended"
)

In [19]:
# function for symmetric embeddings 
def embed_symmetric(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Symmetric)
    result = search_model.semantic_embed(request)
    return result.embedding

# function for asymmetric embeddings of Queries
def embed_query(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Query)
    result = search_model.semantic_embed(request)
    return result.embedding

# function for asymmetric embeddings of Documents
def embed_document(text: str):
    request = SemanticEmbeddingRequest(prompt=Prompt.from_text(text), representation=SemanticRepresentation.Document)
    result = search_model.semantic_embed(request)
    return result.embedding

# function to calculate similarity
def cosine_similarity(v1: Sequence[float], v2: Sequence[float]) -> float:
    "compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)"
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

# function for getting a summary
def generate_summary(text: str):
    request = SummarizationRequest(prompt=Prompt.from_text(text))
    result = model.summarize(request)
    return result.summary

# function that splits text by paragraphs
def split_text(text: str):
    return text.split("\n\n")

# function that evaluate two texts
def evaluate(text1: str, text2: str):
    request = EvaluationRequest(prompt=Prompt.from_text(text1), completion_expected=text2)
    result = model.evaluate(request)
    return result

In [20]:
text_to_summarize = ""
with open("text_to_summarize.txt", "r") as f:
    text_to_summarize = f.read()

### Task 1: 
Create a function that creates a guided summary of a text.
- The function should take a text as input as well as some form of guidance
- Tipp: use multiple functions from the previous exercises
- There are several ways to solve this task. Try to find a solution that works for you.
- Tipp: look at the available functions


In [42]:
# Create a function that creates a guided summary of a text.
def guided_summary(text : str, guidance : str):
    pass

### Task 2:
Create a function that searches a list of texts based on a question and then generates an answer.
- Use search to find the most relevant text
- Use a completion request with your own prompt to generate an answer

In [25]:
texts_to_search = [
    "The judiciary of Poland (Polish: sądownictwo w Polsce) are the authorities exercising the judicial power of the Polish state on the basis of Chapter 8 of the Constitution of Poland.[a] As in almost all countries of continental Europe, the Polish judiciary operates within the framework of civil law.The courts (sądy), designated by the Constitution as those exercising the administration of justice (wymiar sprawiedliwości), are the bodies that review the vast majority of cases, with the exception of those specifically assigned to the two tribunals (trybunały). ",
    "Nicholas Blake 'Nick' Solak (born January 11, 1995) is an American professional baseball second baseman and outfielder for the Texas Rangers of Major League Baseball (MLB). Solak attended Naperville North High School in Naperville, Illinois, and the University of Louisville in Louisville, Kentucky.",
    "Die Buddhistenkrise war ein Zeitraum politisch-religiöser Anspannungen in Südvietnam vom 8. Mai bis 2. November 1963. Sie wurde durch das Verbot der buddhistischen Flagge durch die Regierung Ngô Đình Diệms ausgelöst, und endete mit einem Putsch der Armee der Republik Vietnam, wobei Ngô Đình Diệm festgenommen und später getötet wurde."
]

In [26]:
def search_and_answer(question : str):
    pass